# <font color='red'><ins>Práctica 3 FAA - Clasificación mediante Algoritmos Genéticos</ins></font>
## Alejandro Santorum Varela - alejandro.santorum@estudiante.uam.es
## José Manuel Chacón Aguilera - josem.chacon@estudiante.uam.es

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [5]:
# My modules
from Datos import Datos
import EstrategiaParticionado
import Clasificador

# External modules
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# Important to pre-install tabulate (pip install tabulate)
from tabulate import tabulate

# Init class Classifier
clasificador = Clasificador.Clasificador()

In [6]:
db_titanic = Datos('../ConjuntosDatos/titanic.data', allNominal = True)
n,m = db_titanic.datos.shape
print("Numero de ejemplos:", n,"| Numero de columnas:", m)

datosenc = Datos.encodeDatos(db_titanic.datos)
n,m = datosenc.shape
print("Numero de ejemplos:", n,"| Numero de columnas (ENCODE):", m)

Numero de ejemplos: 1046 | Numero de columnas: 4
Numero de ejemplos: 1046 | Numero de columnas (ENCODE): 22


In [7]:
print(db_titanic.datos[0])
print(datosenc[0])

[1. 1. 3. 1.]
[0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [149]:
# Entrenamiento. Consiste en sacar el individuo "ÓPTIMO".

    # - Definir la función/funciones de cruce
    # - Definir la mutación/mutaciones
    # - Definir la función/funciones de adaptación
    
    # ------------> Si nos calentamos, implementamos varias de estas y las consideramos hiper-parámetros del algoritmo.
    
# Clasificación. Consiste en comparar los genes de tu ejemplo con el individuo obtenido anteriormente, si tiene una codificación
# genética "parecida" codificas como superviviente.
    # Definir que es "parecido". 


In [150]:
# El INIT que tienes implementado no lo comprendo muy bien ni tampoco se usarlo, este solo es de prueba, 
# genera cadenas (sin sentido) para que pueda implementar primeras versiones tanto del cruze como de la mutación
population = 100
aleat = np.zeros((population, m-1))
aleat[:] = np.random.randint(0, 2, (population, m-1))
print (aleat[0])

[1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]


In [151]:
# Esto solo es visual
Range = np.arange(0, m-1)

In [152]:
# CRUCE este es en un punto?????? Que otras variantes podemos implementar?????
# imput
P1 = aleat[0]
P2 = aleat[1]
print(Range)
print(P1)
print(P2)
# algoritmo
cp = np.random.randint(1, m-1)

print("\nPunto de Cruce Aleatorio ---> Pos. %d\n" % (cp))

H1 = np.zeros(m-1)
H1[:cp] = P1[:cp] 
H1[-(m-1-cp):] = P2[-(m-1-cp):]
H2 = np.zeros(m-1)
H2[:cp] = P2[:cp] 
H2[-(m-1-cp):] = P1[-(m-1-cp):]
# output
print(Range)
print(H1)
print(H2)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]
[0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.]

Punto de Cruce Aleatorio ---> Pos. 8

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.]
[0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]


In [153]:
# Mutación, permuta bit en posiciones aleatorias, acepta como hiper-parámetro el número máximo de bits que queremos mutar.
# No compruebo duplicidades, si muta dos veces el mismo gen es como si no lo hubiese hecho, lo dejamos así.

def mutar(IND, max_mute):
    print(Range)
    print(IND)
    n_mute = np.random.randint(1, max_mute+1)
    for i in range(n_mute):
        pos = np.random.randint(0, 21)
        print(pos)
        if(IND[pos] == 1):
            IND[pos] = 0
        else:
            IND[pos] = 1
    print(Range)
    print(IND)
        
    

In [154]:
mutar(aleat[0], 3)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]
10
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]
